
# Recommender Prediction Algorithm Downselect

## Install and Import Libraries
> We are using the Surprise library because it has all the tools we need to import our dataset, <br>
calculate the similarities and errors from all the similarity algorithms, fit the algorithms <br>
to our dataset, and then output the similarity matrix.

In [1]:
!pip install surprise

In [2]:
import json
import time
import pandas as pd
from surprise.model_selection import GridSearchCV
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import SVD
from surprise import SVDpp
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

## Import Preprocessed User, Restaurant, and Review Data

In [3]:
start = time.time()
with open('MA_users.json', encoding="utf8") as fin:
    print('Reading',fin.name)
    user_data = json.load(fin)
end = time.time()
duration_without_dr = end-start
print("Time taken to load user data: %d seconds" %duration_without_dr)


Reading MA_users.json
Time taken to load user data: 0 seconds


In [4]:
start = time.time()
with open('MA_restaurants.json', encoding="utf8") as fin:
    print('Reading',fin.name)
    restaurant_data = json.load(fin)
end = time.time()
duration_without_dr = end-start
print("Time taken to load restaurant data: %d seconds" %duration_without_dr)

Reading MA_restaurants.json
Time taken to load restaurant data: 0 seconds


In [5]:
start = time.time()
with open('MA_reviews.json', encoding="utf8") as fin:
    print('Reading',fin.name)
    review_data = json.load(fin)
end = time.time()
duration_without_dr = end-start
print("Time taken to load review data: %d seconds" %duration_without_dr)

Reading MA_reviews.json
Time taken to load review data: 2 seconds


In [6]:
print(str(len(user_data)) + ' user records loaded')
print(str(len(review_data)) + ' review records loaded')
print(str(len(restaurant_data)) + ' business records loaded')

13046 user records loaded
465276 review records loaded
6249 business records loaded


## Rating Dictionary and Dataframe
Create the rating Dictionary and Convert that to the Main Rating Dataframe

In [7]:
rating_dict = {'user_id': [],
               'business_id': [],
               'rating': []}

for review in review_data:
    rating_dict['user_id'].append(review['user_id'])
    rating_dict['business_id'].append(review['business_id'])
    rating_dict['rating'].append(review['stars'])

df = pd.DataFrame(rating_dict)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['user_id', 'business_id', 'rating']], reader)
print('The data frame shape:\t{}'.format(df.shape))

The data frame shape:	(465276, 3)


In [8]:
if 'df' in globals(): # Clean up memory 
    del df

# Hyper Parameter Tuning
Use GridSearchCV() to optimize each alogrithm and its parameters

## K-Nearest Neighbour (KNN) Based Algorithms
> Set up the parameter grid for the KNN Gridsearch

In [9]:
sim_options = {
    "name": ["msd","cosine"],
    "min_support": [1,5],
    "user_based": [False,True]
}
param_grid = {"k": [10, 40],
              "min_k:": [1, 5, 10],
              "sim_options": sim_options}

### Gridsearch on KNNBasic <br>
> The code is commented out because it takes a long time to run <br>
The results of the Gridsearch are given after the code block

In [10]:
#if 'grid_search' in globals():
#    del grid_search
#grid_search = GridSearchCV(KNNBasic, param_grid, measures = ["rmse", "mae"], cv=5, n_jobs=3,joblib_verbose=10)
#grid_search.fit(data)

In [11]:
#KNNBasic_results_df = pd.DataFrame.from_dict(grid_search.cv_results)
#algo = KNNBasic_results_df
#for n in range(len(algo.params)):
#    print('Mean RSME: {0:11.10f} Mean MAE: {1:11.10f} Params{2:}'
#          .format(algo.mean_test_rmse[n],algo.mean_test_mae[n],algo.params[n]))
#print('\nBest RSME: {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_rmse'] == 1, 'mean_test_rmse'].iloc[0],algo.loc[algo['rank_test_rmse'] == 1, 'params'].iloc[0]))
#print('\nBest MAE:  {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_mae'] == 1, 'mean_test_mae'].iloc[0],algo.loc[algo['rank_test_mae'] == 1, 'params'].iloc[0]))

### KNNBasic Results
 > Best RSME: 1.0797624464 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Best MAE:  0.8460764842 Best Params:{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}

### Gridsearch on KNNWithMeans to find the optimal hyperparameters 

In [12]:
#if 'grid_search' in globals():
#    del grid_search
#grid_search = GridSearchCV(KNNWithMeans, param_grid, measures = ["rmse", "mae"], cv=5, n_jobs=3,joblib_verbose=10)
#grid_search.fit(data)

In [13]:
#KNNMeans_results_df = pd.DataFrame.from_dict(grid_search.cv_results)
#algo = KNNMeans_results_df
#for n in range(len(algo.params)):
#    print('Mean RSME: {0:11.10f} Mean MAE: {1:11.10f} Params{2:}'
#          .format(algo.mean_test_rmse[n],algo.mean_test_mae[n],algo.params[n]))
#print('\nBest RSME: {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_rmse'] == 1, 'mean_test_rmse'].iloc[0],algo.loc[algo['rank_test_rmse'] == 1, 'params'].iloc[0]))
#print('\nBest MAE:  {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_mae'] == 1, 'mean_test_mae'].iloc[0],algo.loc[algo['rank_test_mae'] == 1, 'params'].iloc[0]))

### KNNWithMeans Results
> Best RSME: 1.0367942082 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Best MAE:  0.8085535760 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}

### Gridsearch on KNNZScore 

In [14]:
#if 'grid_search' in globals():
#    del grid_search
#grid_search = GridSearchCV(KNNWithZScore, param_grid, measures = ["rmse", "mae"], cv=5, n_jobs=3,joblib_verbose=10)
#grid_search.fit(data)

In [15]:
#KNNZScore_results_df = pd.DataFrame.from_dict(grid_search.cv_results)
#algo = KNNZScore_results_df
#for n in range(len(algo.params)):
#    print('Mean RSME: {0:11.10f} Mean MAE: {1:11.10f} Params{2:}'
#          .format(algo.mean_test_rmse[n],algo.mean_test_mae[n],algo.params[n]))
#print('\nBest RSME: {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_rmse'] == 1, 'mean_test_rmse'].iloc[0],algo.loc[algo['rank_test_rmse'] == 1, 'params'].iloc[0]))
#print('\nBest MAE:  {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_mae'] == 1, 'mean_test_mae'].iloc[0],algo.loc[algo['rank_test_mae'] == 1, 'params'].iloc[0]))

### KNNZScore Results
> Best RSME: 1.0387852982 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Best MAE:  0.8090497383 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}

## Singular Value Decomposition (SVD) Prediction Algorithms
> Set up the parameter grid for the SVD Gridsearch

In [17]:
sim_options = {
    "n_epochs": [5, 10, 20, 30],
    "lr_all": [.0025, .005, .001, .01]}


### Gridsearch on SVD 

In [18]:
#if 'grid_search' in globals():
#    del grid_search
#grid_search = GridSearchCV(SVD, sim_options, measures = ["rmse", "mae"], cv=5, n_jobs=9,joblib_verbose=10)
#grid_search.fit(data)

In [19]:
#SVD_results_df = pd.DataFrame.from_dict(grid_search.cv_results)
#algo = SVD_results_df
#for n in range(len(algo.params)):
#    print('Mean RSME: {0:11.10f} Mean MAE: {1:11.10f} Params{2:}'
#          .format(algo.mean_test_rmse[n],algo.mean_test_mae[n],algo.params[n]))
#print('\nBest RSME: {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_rmse'] == 1, 'mean_test_rmse'].iloc[0],algo.loc[algo['rank_test_rmse'] == 1, 'params'].iloc[0]))
#print('\nBest MAE:  {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_mae'] == 1, 'mean_test_mae'].iloc[0],algo.loc[algo['rank_test_mae'] == 1, 'params'].iloc[0]))

### SVD Results
> Best RSME: 1.0272185025 Best Params:{'n_epochs': 20, 'lr_all': 0.0025} <br>
Best MAE:  0.8116613978 Best Params:{'n_epochs': 30, 'lr_all': 0.0025}

### Gridsearch on SVD++ 

In [1]:
#if 'grid_search' in globals():
#    del grid_search
#grid_search = GridSearchCV(SVDpp, sim_options, measures = ["rmse", "mae"], cv=5, n_jobs=9,joblib_verbose=10)
#grid_search.fit(data)

In [21]:
#SVDpp_results_df = pd.DataFrame.from_dict(grid_search.cv_results)
#algo = SVDpp_results_df
#for n in range(len(algo.params)):
#    print('Mean RSME: {0:11.10f} Mean MAE: {1:11.10f} Params{2:}'
#          .format(algo.mean_test_rmse[n],algo.mean_test_mae[n],algo.params[n]))
#print('\nBest RSME: {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_rmse'] == 1, 'mean_test_rmse'].iloc[0],algo.loc[algo['rank_test_rmse'] == 1, 'params'].iloc[0]))
#print('\nBest MAE:  {0:11.10f} Best Params:{1:}'
#      .format(algo.loc[algo['rank_test_mae'] == 1, 'mean_test_mae'].iloc[0],algo.loc[algo['rank_test_mae'] == 1, 'params'].iloc[0]))

### SVD++ Results
> Best RSME: 1.0230810561 Best Params:{'n_epochs': 30, 'lr_all': 0.0025} <br>
Best MAE:  0.8075366007 Best Params:{'n_epochs': 30, 'lr_all': 0.0025}

# Cross Validation on Optimized Prediction Algorithms 

## KNNBasic

In [22]:
sim_options = {
    "name": "msd",
    "min_support": 1,
    "user_based": True
}

algo = KNNBasic(k=40, min_k=5, sim_options=sim_options)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=3, verbose=True);

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0782  1.0718  1.0713  1.0748  1.0759  1.0744  0.0026  
MAE (testset)     0.8456  0.8422  0.8419  0.8448  0.8446  0.8438  0.0015  
Fit time          8.86    8.98    8.88    8.33    8.17    8.64    0.33    
Test time         12.38   12.43   12.48   11.96   11.88   12.22   0.25    


## KNNWithMeans

In [23]:
sim_options = {
    "name": "cosine",
    "min_support": 1,
    "user_based": False
}
algo = KNNWithMeans(k=40, min_k=5, sim_options=sim_options)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=3, verbose=True);

Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0309  1.0340  1.0312  1.0305  1.0326  1.0318  0.0013  
MAE (testset)     0.8101  0.8129  0.8091  0.8096  0.8101  0.8103  0.0013  
Fit time          7.79    7.84    7.72    7.47    7.40    7.65    0.18    
Test time         6.21    6.20    6.18    6.13    6.06    6.16    0.06    


## KNNWithZScore

In [24]:
sim_options = {
    "name": "cosine",
    "min_support": 1,
    "user_based": False
}
algo = KNNWithZScore(k=40, min_k=5, sim_options=sim_options)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=3, verbose=True);

Evaluating RMSE, MAE of algorithm KNNWithZScore on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0288  1.0331  1.0324  1.0350  1.0323  1.0323  0.0020  
MAE (testset)     0.8072  0.8114  0.8088  0.8113  0.8085  0.8095  0.0017  
Fit time          8.01    8.05    8.04    7.63    7.58    7.86    0.21    
Test time         6.66    6.80    6.76    6.58    6.70    6.70    0.08    


## SVD

In [25]:

algo = SVD(n_epochs=20, lr_all=0.0025)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=9, verbose=True);

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0231  1.0241  1.0225  1.0206  1.0226  1.0226  0.0011  
MAE (testset)     0.8106  0.8105  0.8087  0.8069  0.8097  0.8093  0.0014  
Fit time          16.84   16.69   16.94   16.84   16.91   16.84   0.08    
Test time         0.63    0.63    0.64    0.64    0.60    0.63    0.02    


## SVD++

In [26]:
algo = SVDpp(n_epochs=30, lr_all=0.0025)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, n_jobs=9, verbose=True);

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0139  1.0226  1.0186  1.0186  1.0192  1.0186  0.0028  
MAE (testset)     0.8001  0.8066  0.8037  0.8045  0.8033  0.8036  0.0021  
Fit time          392.62  404.84  393.37  397.35  396.86  397.01  4.33    
Test time         4.64    4.35    4.59    4.50    4.46    4.51    0.10    


## Cross Validation Results Summary
<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-q2ov{border-color:#000000;font-family:Arial, Helvetica, sans-serif !important;;text-align:left;vertical-align:top}
.tg .tg-xesh{border-color:#000000;font-family:Arial, Helvetica, sans-serif !important;;text-align:left;vertical-align:middle}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-q2ov"><span style="font-weight:bold">Predictor Algorithm</span></th>
    <th class="tg-q2ov"><span style="font-weight:bold">Average RMSE</span></th>
    <th class="tg-q2ov"><span style="font-weight:bold">Average MAE</span></th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-q2ov">KNN Basic</td>
    <td class="tg-q2ov">1.0744</td>
    <td class="tg-q2ov">0.8438</td>
  </tr>
  <tr>
    <td class="tg-q2ov">KNN Means</td>
    <td class="tg-xesh">1.0318</td>
    <td class="tg-xesh">0.8103</td>
  </tr>
  <tr>
    <td class="tg-q2ov">KNN ZScore</td>
    <td class="tg-xesh">1.0323</td>
    <td class="tg-xesh">0.8095</td>
  </tr>
  <tr>
    <td class="tg-q2ov"></td>
    <td class="tg-q2ov"></td>
    <td class="tg-q2ov"></td>
  </tr>
  <tr>
    <td class="tg-q2ov">SVD</td>
    <td class="tg-xesh">1.0226</td>
    <td class="tg-xesh">0.8093</td>
  </tr>
  <tr>
    <td class="tg-q2ov">SVD++</td>
    <td class="tg-xesh">1.0186</td>
    <td class="tg-xesh">0.8036</td>
  </tr>
</tbody>
</table>

# Conclusion
The best predictor algorithm based on Average RMSE and MAE is SVD++.  As such, SVD++ is our chosen prediction algorithm for the final recommender of "Yelp On the Go!"

# Backup Data from Runs

## KNNBasic
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers. <br>
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    7.9s <br>
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:   32.7s <br>
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   52.2s <br>
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:  1.5min <br>
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  2.4min <br>
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  3.3min <br>
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  4.2min <br>
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed:  4.9min <br>
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:  6.4min <br>
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed:  7.9min <br>
[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed:  8.8min <br>
[Parallel(n_jobs=3)]: Done 107 tasks      | elapsed: 10.5min <br>
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed: 12.2min <br>
[Parallel(n_jobs=3)]: Done 139 tasks      | elapsed: 13.7min <br>
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed: 16.1min <br>
[Parallel(n_jobs=3)]: Done 175 tasks      | elapsed: 17.6min <br>
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 20.1min <br>
[Parallel(n_jobs=3)]: Done 215 tasks      | elapsed: 22.1min <br>
[Parallel(n_jobs=3)]: Done 240 out of 240 | elapsed: 25.5min finished  <br>

Mean RMSE: 1.1207483198 Mean MAE: 0.8704986843 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}  <br>
Mean RMSE: 1.1134275198 Mean MAE: 0.8676869132 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>
Mean RMSE: 1.1579731854 Mean MAE: 0.8879868382 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}  <br>
Mean RMSE: 1.1483985917 Mean MAE: 0.9015830440 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}  <br>
Mean RMSE: 1.1317509528 Mean MAE: 0.8901500859 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}  <br>
Mean RMSE: 1.1185394953 Mean MAE: 0.8864770112 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}  <br>
Mean RMSE: 1.1678848949 Mean MAE: 0.8979010320 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}  <br>
Mean RMSE: 1.1553289835 Mean MAE: 0.9092083362 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}}  <br>
Mean RMSE: 1.1207483198 Mean MAE: 0.8704986843 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}  <br>
Mean RMSE: 1.1134275198 Mean MAE: 0.8676869132 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>
Mean RMSE: 1.1579731854 Mean MAE: 0.8879868382 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}  <br>
Mean RMSE: 1.1483985917 Mean MAE: 0.9015830440 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}  <br>
Mean RMSE: 1.1317509528 Mean MAE: 0.8901500859 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}  <br>
Mean RMSE: 1.1185394953 Mean MAE: 0.8864770112 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}  <br>
Mean RMSE: 1.1678848949 Mean MAE: 0.8979010320 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}  <br>
Mean RMSE: 1.1553289835 Mean MAE: 0.9092083362 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}}  <br>
Mean RMSE: 1.1207483198 Mean MAE: 0.8704986843 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}  <br>
Mean RMSE: 1.1134275198 Mean MAE: 0.8676869132 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>
Mean RMSE: 1.1579731854 Mean MAE: 0.8879868382 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}  <br>
Mean RMSE: 1.1483985917 Mean MAE: 0.9015830440 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}  <br>
Mean RMSE: 1.1317509528 Mean MAE: 0.8901500859 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}  <br>
Mean RMSE: 1.1185394953 Mean MAE: 0.8864770112 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}  <br>
Mean RMSE: 1.1678848949 Mean MAE: 0.8979010320 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}  <br>
Mean RMSE: 1.1553289835 Mean MAE: 0.9092083362 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}}  <br>
Mean RMSE: 1.0988640938 Mean MAE: 0.8584146262 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}  <br>
Mean RMSE: 1.0797624464 Mean MAE: 0.8460764842 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>
Mean RMSE: 1.1519888017 Mean MAE: 0.8850846700 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}  <br>
Mean RMSE: 1.1424616886 Mean MAE: 0.8975418082 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}  <br>
Mean RMSE: 1.1064197209 Mean MAE: 0.8716636523 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}  <br>
Mean RMSE: 1.0854190414 Mean MAE: 0.8598299793 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}  <br>
Mean RMSE: 1.1583798093 Mean MAE: 0.8930909435 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}  <br>
Mean RMSE: 1.1484190091 Mean MAE: 0.9044068678 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}}  <br>
Mean RMSE: 1.0988640938 Mean MAE: 0.8584146262 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}  <br>
Mean RMSE: 1.0797624464 Mean MAE: 0.8460764842 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>
Mean RMSE: 1.1519888017 Mean MAE: 0.8850846700 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}  <br>
Mean RMSE: 1.1424616886 Mean MAE: 0.8975418082 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}  <br>
Mean RMSE: 1.1064197209 Mean MAE: 0.8716636523 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}  <br>
Mean RMSE: 1.0854190414 Mean MAE: 0.8598299793 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}  <br>
Mean RMSE: 1.1583798093 Mean MAE: 0.8930909435 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}  <br>
Mean RMSE: 1.1484190091 Mean MAE: 0.9044068678 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}}  <br>
Mean RMSE: 1.0988640938 Mean MAE: 0.8584146262 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}  <br>
Mean RMSE: 1.0797624464 Mean MAE: 0.8460764842 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>
Mean RMSE: 1.1519888017 Mean MAE: 0.8850846700 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}}  <br>
Mean RMSE: 1.1424616886 Mean MAE: 0.8975418082 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}}  <br>
Mean RMSE: 1.1064197209 Mean MAE: 0.8716636523 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}  <br>
Mean RMSE: 1.0854190414 Mean MAE: 0.8598299793 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}}  <br>
Mean RMSE: 1.1583798093 Mean MAE: 0.8930909435 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}}  <br>
Mean RMSE: 1.1484190091 Mean MAE: 0.9044068678 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}}  <br>  <br>

Best RMSE: 1.0797624464 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}  <br>  <br>

Best MAE:  0.8460764842 Best Params:{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}}

## KNNMeans
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers. <br>
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    7.7s <br>
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:   33.4s <br>
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   52.8s <br>
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:  1.5min <br>
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  2.5min <br>
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  3.3min <br>
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  4.2min <br>
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed:  4.9min <br>
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:  6.4min <br>
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed:  7.8min <br>
[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed:  8.7min <br>
[Parallel(n_jobs=3)]: Done 107 tasks      | elapsed: 10.4min <br>
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed: 12.0min <br>
[Parallel(n_jobs=3)]: Done 139 tasks      | elapsed: 13.6min <br>
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed: 15.9min <br>
[Parallel(n_jobs=3)]: Done 175 tasks      | elapsed: 17.4min <br>
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 19.9min <br>
[Parallel(n_jobs=3)]: Done 215 tasks      | elapsed: 21.8min <br>
[Parallel(n_jobs=3)]: Done 240 out of 240 | elapsed: 25.3min finished <br>  <br>

Mean RMSE: 1.0655983535 Mean MAE: 0.8247306086 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0810146844 Mean MAE: 0.8394403437 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0969267715 Mean MAE: 0.8441156753 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1072333151 Mean MAE: 0.8583919258 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0588264076 Mean MAE: 0.8252494764 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0756201468 Mean MAE: 0.8414108802 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0939136988 Mean MAE: 0.8440246289 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1061819101 Mean MAE: 0.8593716276 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0655983535 Mean MAE: 0.8247306086 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0810146844 Mean MAE: 0.8394403437 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0969267715 Mean MAE: 0.8441156753 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1072333151 Mean MAE: 0.8583919258 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0588264076 Mean MAE: 0.8252494764 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0756201468 Mean MAE: 0.8414108802 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0939136988 Mean MAE: 0.8440246289 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1061819101 Mean MAE: 0.8593716276 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0655983535 Mean MAE: 0.8247306086 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0810146844 Mean MAE: 0.8394403437 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0969267715 Mean MAE: 0.8441156753 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1072333151 Mean MAE: 0.8583919258 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0588264076 Mean MAE: 0.8252494764 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0756201468 Mean MAE: 0.8414108802 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0939136988 Mean MAE: 0.8440246289 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1061819101 Mean MAE: 0.8593716276 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0418043176 Mean MAE: 0.8085535760 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0505257691 Mean MAE: 0.8166440026 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0863256661 Mean MAE: 0.8361652569 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.0991071862 Mean MAE: 0.8512321919 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0367942082 Mean MAE: 0.8087903210 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0464099210 Mean MAE: 0.8180279348 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0841756143 Mean MAE: 0.8367188777 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.0989079211 Mean MAE: 0.8526456971 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0418043176 Mean MAE: 0.8085535760 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0505257691 Mean MAE: 0.8166440026 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0863256661 Mean MAE: 0.8361652569 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.0991071862 Mean MAE: 0.8512321919 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0367942082 Mean MAE: 0.8087903210 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0464099210 Mean MAE: 0.8180279348 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0841756143 Mean MAE: 0.8367188777 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.0989079211 Mean MAE: 0.8526456971 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0418043176 Mean MAE: 0.8085535760 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0505257691 Mean MAE: 0.8166440026 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0863256661 Mean MAE: 0.8361652569 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.0991071862 Mean MAE: 0.8512321919 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0367942082 Mean MAE: 0.8087903210 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0464099210 Mean MAE: 0.8180279348 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0841756143 Mean MAE: 0.8367188777 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.0989079211 Mean MAE: 0.8526456971 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br> <br>

Best RMSE: 1.0367942082 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br> <br>

Best MAE:  0.8085535760 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}}

## KNNZScore
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers. <br>
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    8.9s <br>
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:   37.7s <br>
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   58.3s <br>
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:  1.6min <br>
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:  2.6min <br>
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  3.6min <br>
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  4.5min <br>
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed:  5.3min <br>
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:  6.9min <br>
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed:  8.4min <br>
[Parallel(n_jobs=3)]: Done  92 tasks      | elapsed:  9.4min <br>
[Parallel(n_jobs=3)]: Done 107 tasks      | elapsed: 11.2min <br>
[Parallel(n_jobs=3)]: Done 122 tasks      | elapsed: 12.9min <br>
[Parallel(n_jobs=3)]: Done 139 tasks      | elapsed: 14.6min <br>
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed: 17.1min <br>
[Parallel(n_jobs=3)]: Done 175 tasks      | elapsed: 18.7min <br>
[Parallel(n_jobs=3)]: Done 194 tasks      | elapsed: 21.4min <br>
[Parallel(n_jobs=3)]: Done 215 tasks      | elapsed: 23.5min <br>
[Parallel(n_jobs=3)]: Done 240 out of 240 | elapsed: 27.1min finished <br> <br>

Mean RMSE: 1.0683386376 Mean MAE: 0.8253927641 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0859524368 Mean MAE: 0.8355716867 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.1045790112 Mean MAE: 0.8476680557 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1243669356 Mean MAE: 0.8624770596 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0607615892 Mean MAE: 0.8255806361 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0767696238 Mean MAE: 0.8356348845 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.1016096449 Mean MAE: 0.8474963164 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1218763154 Mean MAE: 0.8624511027 Params{'k': 10, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0683386376 Mean MAE: 0.8253927641 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0859524368 Mean MAE: 0.8355716867 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.1045790112 Mean MAE: 0.8476680557 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1243669356 Mean MAE: 0.8624770596 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0607615892 Mean MAE: 0.8255806361 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0767696238 Mean MAE: 0.8356348845 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.1016096449 Mean MAE: 0.8474963164 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1218763154 Mean MAE: 0.8624511027 Params{'k': 10, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0683386376 Mean MAE: 0.8253927641 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0859524368 Mean MAE: 0.8355716867 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.1045790112 Mean MAE: 0.8476680557 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1243669356 Mean MAE: 0.8624770596 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0607615892 Mean MAE: 0.8255806361 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0767696238 Mean MAE: 0.8356348845 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.1016096449 Mean MAE: 0.8474963164 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1218763154 Mean MAE: 0.8624511027 Params{'k': 10, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0443347945 Mean MAE: 0.8090751138 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0552517612 Mean MAE: 0.8141773828 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0940389812 Mean MAE: 0.8396742721 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1150317967 Mean MAE: 0.8549600818 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0387852982 Mean MAE: 0.8090497383 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0479187969 Mean MAE: 0.8144749330 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0914201809 Mean MAE: 0.8397484110 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1133504557 Mean MAE: 0.8552410299 Params{'k': 40, 'min_k:': 1, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0443347945 Mean MAE: 0.8090751138 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0552517612 Mean MAE: 0.8141773828 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0940389812 Mean MAE: 0.8396742721 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1150317967 Mean MAE: 0.8549600818 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0387852982 Mean MAE: 0.8090497383 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0479187969 Mean MAE: 0.8144749330 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0914201809 Mean MAE: 0.8397484110 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1133504557 Mean MAE: 0.8552410299 Params{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0443347945 Mean MAE: 0.8090751138 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0552517612 Mean MAE: 0.8141773828 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0940389812 Mean MAE: 0.8396742721 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1150317967 Mean MAE: 0.8549600818 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'msd', 'min_support': 5, 'user_based': True}} <br>
Mean RMSE: 1.0387852982 Mean MAE: 0.8090497383 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>
Mean RMSE: 1.0479187969 Mean MAE: 0.8144749330 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': True}} <br>
Mean RMSE: 1.0914201809 Mean MAE: 0.8397484110 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': False}} <br>
Mean RMSE: 1.1133504557 Mean MAE: 0.8552410299 Params{'k': 40, 'min_k:': 10, 'sim_options': {'name': 'cosine', 'min_support': 5, 'user_based': True}} <br> <br>

Best RMSE: 1.0387852982 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}} <br>

Best MAE:  0.8090497383 Best Params:{'k': 40, 'min_k:': 5, 'sim_options': {'name': 'cosine', 'min_support': 1, 'user_based': False}}

## SVD
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers. <br>
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:    6.0s <br>
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:   15.5s <br>
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:   21.5s <br>
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:   35.3s <br>
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:   53.7s <br>
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:  1.3min <br>
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:  2.1min <br>
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed:  3.0min <br>
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed:  4.4min <br>
[Parallel(n_jobs=3)]: Done  80 out of  80 | elapsed:  6.4min finished <br>

Mean RMSE: 1.0553292065 Mean MAE: 0.8416993990 Params{'n_epochs': 5, 'lr_all': 0.0025} <br>
Mean RMSE: 1.0370958851 Mean MAE: 0.8244407821 Params{'n_epochs': 5, 'lr_all': 0.005} <br>
Mean RMSE: 1.0850222425 Mean MAE: 0.8707043753 Params{'n_epochs': 5, 'lr_all': 0.001} <br>
Mean RMSE: 1.0277610396 Mean MAE: 0.8138482030 Params{'n_epochs': 5, 'lr_all': 0.01} <br>
Mean RMSE: 1.0374359618 Mean MAE: 0.8247214879 Params{'n_epochs': 10, 'lr_all': 0.0025} <br>
Mean RMSE: 1.0273846146 Mean MAE: 0.8137289083 Params{'n_epochs': 10, 'lr_all': 0.005} <br>
Mean RMSE: 1.0621503302 Mean MAE: 0.8481700594 Params{'n_epochs': 10, 'lr_all': 0.001} <br>
Mean RMSE: 1.0319438457 Mean MAE: 0.8132544921 Params{'n_epochs': 10, 'lr_all': 0.01} <br>
Mean RMSE: 1.0272185025 Mean MAE: 0.8134268693 Params{'n_epochs': 20, 'lr_all': 0.0025} <br>
Mean RMSE: 1.0321072951 Mean MAE: 0.8134278075 Params{'n_epochs': 20, 'lr_all': 0.005} <br>
Mean RMSE: 1.0422773009 Mean MAE: 0.8292470705 Params{'n_epochs': 20, 'lr_all': 0.001} <br>
Mean RMSE: 1.0711510227 Mean MAE: 0.8384373243 Params{'n_epochs': 20, 'lr_all': 0.01} <br>
Mean RMSE: 1.0273488208 Mean MAE: 0.8116613978 Params{'n_epochs': 30, 'lr_all': 0.0025} <br>
Mean RMSE: 1.0505430556 Mean MAE: 0.8246282391 Params{'n_epochs': 30, 'lr_all': 0.005} <br>
Mean RMSE: 1.0335123672 Mean MAE: 0.8208200307 Params{'n_epochs': 30, 'lr_all': 0.001} <br>
Mean RMSE: 1.0928715037 Mean MAE: 0.8520616481 Params{'n_epochs': 30, 'lr_all': 0.01} <br>

Best RMSE: 1.0272185025 Best Params:{'n_epochs': 20, 'lr_all': 0.0025} <br>
Best MAE:  0.8116613978 Best Params:{'n_epochs': 30, 'lr_all': 0.0025}

## SVD++
[Parallel(n_jobs=9)]: Using backend LokyBackend with 9 concurrent workers. <br>
[Parallel(n_jobs=9)]: Done   7 tasks      | elapsed:  1.5min <br>
[Parallel(n_jobs=9)]: Done  14 tasks      | elapsed:  3.0min <br>
[Parallel(n_jobs=9)]: Done  23 tasks      | elapsed:  5.8min <br>
[Parallel(n_jobs=9)]: Done  32 tasks      | elapsed:  8.5min <br>
[Parallel(n_jobs=9)]: Done  43 tasks      | elapsed: 13.8min <br>
[Parallel(n_jobs=9)]: Done  54 tasks      | elapsed: 19.2min <br>
[Parallel(n_jobs=9)]: Done  72 out of  80 | elapsed: 35.5min remaining:  3.9min <br>
[Parallel(n_jobs=9)]: Done  80 out of  80 | elapsed: 42.0min finished <br>

Mean RMSE: 1.0527578934 Mean MAE: 0.8395470539 Params{'n_epochs': 5, 'lr_all': 0.0025} <br>
Mean RMSE: 1.0350319500 Mean MAE: 0.8223487040 Params{'n_epochs': 5, 'lr_all': 0.005} <br>
Mean RMSE: 1.0817778398 Mean MAE: 0.8688821281 Params{'n_epochs': 5, 'lr_all': 0.001} <br>
Mean RMSE: 1.0251171408 Mean MAE: 0.8113078296 Params{'n_epochs': 5, 'lr_all': 0.01} <br>
Mean RMSE: 1.0349418847 Mean MAE: 0.8222710249 Params{'n_epochs': 10, 'lr_all': 0.0025} <br>
Mean RMSE: 1.0246375321 Mean MAE: 0.8110269266 Params{'n_epochs': 10, 'lr_all': 0.005} <br>
Mean RMSE: 1.0595689547 Mean MAE: 0.8462357504 Params{'n_epochs': 10, 'lr_all': 0.001} <br>
Mean RMSE: 1.0258545384 Mean MAE: 0.8081739308 Params{'n_epochs': 10, 'lr_all': 0.01} <br>
Mean RMSE: 1.0245282081 Mean MAE: 0.8109067178 Params{'n_epochs': 20, 'lr_all': 0.0025} <br>
Mean RMSE: 1.0252212612 Mean MAE: 0.8076680217 Params{'n_epochs': 20, 'lr_all': 0.005} <br>
Mean RMSE: 1.0400221067 Mean MAE: 0.8271907348 Params{'n_epochs': 20, 'lr_all': 0.001} <br>
Mean RMSE: 1.0643620756 Mean MAE: 0.8319213222 Params{'n_epochs': 20, 'lr_all': 0.01} <br>
Mean RMSE: 1.0230810561 Mean MAE: 0.8075366007 Params{'n_epochs': 30, 'lr_all': 0.0025} <br>
Mean RMSE: 1.0405533039 Mean MAE: 0.8166455465 Params{'n_epochs': 30, 'lr_all': 0.005} <br>
Mean RMSE: 1.0314976872 Mean MAE: 0.8187010611 Params{'n_epochs': 30, 'lr_all': 0.001} <br>
Mean RMSE: 1.1138983438 Mean MAE: 0.8647625829 Params{'n_epochs': 30, 'lr_all': 0.01} <br>

Best RMSE: 1.0230810561 Best Params:{'n_epochs': 30, 'lr_all': 0.0025} <br>
Best MAE:  0.8075366007 Best Params:{'n_epochs': 30, 'lr_all': 0.0025} <br>